In [1]:
import pandas
from scipy import stats
import numpy
pandas.set_option('display.max_columns', None)

In [2]:
data = pandas.read_csv("./data/all_consolidated_raw.csv")

In [3]:
data.sample(3)

,player_season_team,2P%,2PA_per_100poss,2PA_per_36min,2PA_per_game,2P_per_100poss,2P_per_36min,2P_per_game,3P%,3PAR_advanced,3PA_per_100poss,3PA_per_36min,3PA_per_game,3P_per_100poss,3P_per_36min,3P_per_game,AGE,AST%_advanced,AST_per_100poss,AST_per_36min,AST_per_game,BLK%_advanced,BLK_per_100poss,BLK_per_36min,BLK_per_game,BPM_advanced,DBPM_advanced,DRB%_advanced,DRB_per_100poss,DRB_per_36min,DRB_per_game,DRTG_per_100poss,DWS_advanced,EFG%_per_game,FG%,FGA_per_100poss,FGA_per_36min,FGA_per_game,FG_per_100poss,FG_per_36min,FG_per_game,FT%,FTA_per_100poss,FTA_per_36min,FTA_per_game,FTR_advanced,FT_per_100poss,FT_per_36min,FT_per_game,G,GS,MP,OBPM_advanced,ORB%_advanced,ORB_per_100poss,ORB_per_36min,ORB_per_game,ORTG_per_100poss,OWS_advanced,PER_advanced,PF_per_100poss,PF_per_36min,PF_per_game,PLAYER,POS,PTS_per_100poss,PTS_per_36min,PTS_per_game,SEASON,STL%_advanced,STL_per_100poss,STL_per_36min,STL_per_game,TEAM,TOV%_advanced,TOV_per_100poss,TOV_per_36min,TOV_per_game,TRB%_advanced,TRB_per_100poss,TRB_per_36min,TRB_per_game,TS%_advanced,USG%_advanced,VORP_advanced,WS/48_advanced,WS_advanced,MVP_VOTES_SHARE,MVP_WINNER,MVP_PODIUM,MVP_CANDIDATE,TEAM_SEASON,W,L,W/L%,GB,PW,PL,PS/G,PA/G,CONF,CONF_RANK
13824,MichaelBeasley_2009_MIA,0.478,23.3,15.7,10.8,11.1,7.5,5.2,0.407,0.085,2.2,1.5,1.0,0.9,0.6,0.4,20,7.7,2.2,1.5,1.0,1.5,1.0,0.7,0.5,-1.7,-1.5,19.8,8.7,5.9,4.1,108.0,2.2,0.490,0.472,25.4,17.1,11.8,12.0,8.1,5.6,0.772,6.4,4.3,3.0,0.252,4.9,3.3,2.3,81,19.0,24.8,-0.2,6.5,2.9,2.0,1.4,103.0,1.2,17.2,4.9,3.3,2.3,MichaelBeasley,PF,29.8,20.1,13.9,2009,1.1,1.1,0.7,0.5,MIA,10.2,3.2,2.2,1.5,13.1,11.7,7.9,5.4,0.528,27.7,0.1,0.081,3.4,0.0,False,False,False,MIA_2009,43,39,0.524,16.0,42,40,98.3,98.0,EASTERN_CONF,8
10353,JarenJackson_2002_ORL,0.471,6.1,4.2,1.9,2.9,2.0,0.9,0.350,0.541,7.2,5.0,2.2,2.5,1.7,0.8,34,8.2,2.9,2.0,0.9,0.0,0.0,0.0,0.0,-3.7,0.0,12.5,5.7,4.0,1.8,107.0,0.1,0.500,0.405,13.2,9.2,4.1,5.4,3.7,1.7,0.500,1.4,1.0,0.4,0.108,0.7,0.5,0.2,9,0.0,16.0,-3.7,0.8,0.4,0.2,0.1,91.0,-0.1,7.2,4.6,3.2,1.4,JarenJackson,SG,13.9,9.7,4.3,2002,1.8,1.8,1.2,0.6,ORL,17.1,2.9,2.0,0.9,6.6,6.1,4.2,1.9,0.503,14.5,-0.1,0.015,0.0,0.0,False,False,False,ORL_2002,44,38,0.537,8.0,46,36,100.5,98.9,EASTERN_CONF,6
6732,RichKing_1994_SEA,0.455,21.3,15.2,1.2,9.7,6.9,0.6,0.000,0.029,0.6,0.5,0.0,0.0,0.0,0.0,24,15.7,5.2,3.7,0.3,1.8,1.3,0.9,0.1,-5.8,-1.4,16.8,7.1,5.1,0.4,104.0,0.1,0.441,0.441,22.0,15.7,1.3,9.7,6.9,0.6,0.500,14.2,10.2,0.8,0.647,7.1,5.1,0.4,27,0.0,2.9,-4.3,13.6,5.8,4.2,0.3,99.0,0.0,12.9,11.6,8.3,0.7,RichKing,C,26.5,18.9,1.5,1994,0.6,0.6,0.5,0.0,SEA,13.8,4.5,3.2,0.3,15.2,12.9,9.2,0.7,0.469,27.8,-0.1,0.070,0.1,0.0,False,False,False,SEA_1994,63,19,0.768,0.0,64,18,105.9,96.9,WESTERN_CONF,3


## 1. Fill NAN

In [4]:
len(data)

20197

In [5]:
def safe_drop(dataframe, cols):
    cols = [col for col in cols if col in dataframe.columns]
    dataframe = dataframe.drop(cols, axis='columns', inplace=False)
    return dataframe

In [6]:
def drop_columns_by_nan_ratio(dataframe, ratio:float):
    if not 0. <= ratio <= 1.:
        raise ValueError("Ratio must be between 0 and 1")
    else:
        ratios = data.isna().sum(axis = 'index') / len(data)
        ratios = ratios[ratios > ratio]
        cols = ratios.index.tolist()
        print("Dropping", cols, "...")
        dataframe = dataframe.drop(cols, axis='columns', inplace=False)
        return dataframe

In [7]:
data.isna().sum(axis = 'index').sort_values(ascending=False).head(10)

GS                  2175
TOV%_advanced       1073
ORTG_per_100poss    1057
USG%_advanced       1008
TOV_per_36min       1008
TOV_per_100poss     1008
TOV_per_game        1003
FT%                  854
2P%                  152
FG%                   98
dtype: int64

In [8]:
data = drop_columns_by_nan_ratio(data, 0.1)

Dropping ['GS'] ...


In [9]:
data["MVP_PODIUM"] = data["MVP_PODIUM"].fillna(False)
data["MVP_CANDIDATE"] = data["MVP_CANDIDATE"].fillna(False)
data["MVP_WINNER"] = data["MVP_WINNER"].fillna(False)
data["MVP_VOTES_SHARE"] = data["MVP_VOTES_SHARE"].fillna(0.)

In [10]:
data.isna().sum(axis = 'index').sort_values(ascending=False).head(10)

TOV%_advanced       1073
ORTG_per_100poss    1057
USG%_advanced       1008
TOV_per_36min       1008
TOV_per_100poss     1008
TOV_per_game        1003
FT%                  854
2P%                  152
FTR_advanced          98
FG%                   98
dtype: int64

In [11]:
cols_with_nan_all = data.isna().sum(axis = 'index').sort_values(ascending=False)
cols_with_nan_all = cols_with_nan_all[cols_with_nan_all>0]
cols_with_nan_all = cols_with_nan_all.index.tolist()

mvp_candidates = data[data["MVP_CANDIDATE"]==True]

cols_with_nan_candidates = mvp_candidates.isna().sum(axis = 'index').sort_values(ascending=False)
cols_with_nan_candidates = cols_with_nan_candidates[cols_with_nan_candidates>0]
cols_with_nan_candidates = cols_with_nan_candidates.index.tolist()

for col in cols_with_nan_all:
    if col not in cols_with_nan_candidates:
        print("Dropping NAN on col", col, "because no MVP candidates have NAN...")
        data = data.dropna(subset=[col], axis='index')
    else:
        print("Not dropping NAN on col", col, "because some MVPs candidates have NAN too")

Not dropping NAN on col TOV%_advanced because some MVPs candidates have NAN too
Not dropping NAN on col ORTG_per_100poss because some MVPs candidates have NAN too
Not dropping NAN on col USG%_advanced because some MVPs candidates have NAN too
Not dropping NAN on col TOV_per_36min because some MVPs candidates have NAN too
Not dropping NAN on col TOV_per_100poss because some MVPs candidates have NAN too
Not dropping NAN on col TOV_per_game because some MVPs candidates have NAN too
Dropping NAN on col FT% because no MVP candidates have NAN...
Dropping NAN on col 2P% because no MVP candidates have NAN...
Dropping NAN on col FTR_advanced because no MVP candidates have NAN...
Dropping NAN on col FG% because no MVP candidates have NAN...
Dropping NAN on col EFG%_per_game because no MVP candidates have NAN...
Dropping NAN on col TS%_advanced because no MVP candidates have NAN...
Dropping NAN on col ORB_per_100poss because no MVP candidates have NAN...
Dropping NAN on col PF_per_100poss because

In [12]:
data.isna().sum(axis = 'index').sort_values(ascending=False).head(10)

TOV_per_100poss     982
ORTG_per_100poss    982
TOV_per_36min       982
TOV_per_game        982
USG%_advanced       982
TOV%_advanced       982
FG_per_game           0
FG_per_36min          0
BLK_per_game          0
FTA_per_game          0
dtype: int64

In [13]:
data[data.isna().any(axis="columns")].SEASON.unique()

array([1974, 1975, 1976, 1977], dtype=int64)

In [14]:
#We will infer values intead of removing years
#data = data[data.SEASON > 1977]

In [15]:
cols_to_fill = data.isna().sum(axis = 'index')
cols_to_fill = cols_to_fill[cols_to_fill>0]
cols_to_fill = cols_to_fill.index.tolist()

# Median of the same position depending on MVP candidate caliber

for col in cols_to_fill:
    print("Filling", col, "...")
    for is_mvp_candidate in True, False:
        for position in data.POS.unique():
            reference = data[(data["MVP_CANDIDATE"]==is_mvp_candidate) & (data["POS"]==position)][col]
            reference_median = reference[reference.notna()].median()
            print("Stat :", col, "| Position :", position, "| MVP candidate :", is_mvp_candidate, " | Median :", reference_median)
            data.loc[(data["MVP_CANDIDATE"]==is_mvp_candidate) & (data["POS"]==position) & (data[col].isna()), col] = reference_median

Filling ORTG_per_100poss ...
Stat : ORTG_per_100poss | Position : C | MVP candidate : True  | Median : 113.0
Stat : ORTG_per_100poss | Position : PF | MVP candidate : True  | Median : 113.0
Stat : ORTG_per_100poss | Position : SG | MVP candidate : True  | Median : 114.0
Stat : ORTG_per_100poss | Position : SF | MVP candidate : True  | Median : 114.0
Stat : ORTG_per_100poss | Position : PG | MVP candidate : True  | Median : 115.0
Stat : ORTG_per_100poss | Position : C | MVP candidate : False  | Median : 104.0
Stat : ORTG_per_100poss | Position : PF | MVP candidate : False  | Median : 105.0
Stat : ORTG_per_100poss | Position : SG | MVP candidate : False  | Median : 103.0
Stat : ORTG_per_100poss | Position : SF | MVP candidate : False  | Median : 104.0
Stat : ORTG_per_100poss | Position : PG | MVP candidate : False  | Median : 103.0
Filling TOV%_advanced ...
Stat : TOV%_advanced | Position : C | MVP candidate : True  | Median : 13.2
Stat : TOV%_advanced | Position : PF | MVP candidate : T

In [16]:
data.isna().sum(axis = 'index').sort_values(ascending=False).head(10)

CONF_RANK           0
FGA_per_36min       0
DBPM_advanced       0
DRB%_advanced       0
DRB_per_100poss     0
DRB_per_36min       0
DRB_per_game        0
DRTG_per_100poss    0
DWS_advanced        0
EFG%_per_game       0
dtype: int64

## 2. Remove outliers

In [17]:
def pretty_print_df_values(dataframe_values):
    entries = []
    for v in dataframe_values:
        entry = " :: ".join([str(vi) for vi in v])
        entries.append(entry)
    print("\n".join(entries))

In [18]:
len(data)

19318

In [19]:
for season in data.SEASON.unique():
    for col in data.columns:
        if col not in ["MVP_VOTES_SHARE", "MVP_WINNER", "MVP_PODIUM", "MVP_CANDIDATE"]:
            try:
                outliers = data[(stats.zscore(data[col]) > 7) & (data.SEASON==season)][["player_season_team", col, "MVP_CANDIDATE"]]
                outliers_mvp_candidates = outliers[outliers["MVP_CANDIDATE"]==True][["player_season_team", col]]
                outliers_non_mvp_candidates = outliers[outliers["MVP_CANDIDATE"]==False][["player_season_team", col]]
                if len(outliers) > 0:
                    print()
                    print("Outliers for", col, "in season", season, ":")
                    pretty_print_df_values(outliers.values)
                if len(outliers_non_mvp_candidates) > 0:
                    invalid_id = outliers_non_mvp_candidates.index.tolist()
                    data = data.loc[~data.index.isin(invalid_id)]
            except TypeError:
                pass


Outliers for BLK_per_game in season 1974 :
ElmoreSmith_1974_LAL :: 4.9 :: False

Outliers for FTR_advanced in season 1974 :
TomPatterson_1974_CAP :: 2.0 :: False

Outliers for TS%_advanced in season 1975 :
WaltBellamy_1975_NOJ :: 1.042 :: False

Outliers for BLK_per_game in season 1976 :
KareemAbdulJabbar_1976_LAL :: 4.1 :: True

Outliers for FTA_per_36min in season 1976 :
WaltWesley_1976_LAL :: 20.6 :: False

Outliers for FTA_per_100poss in season 1977 :
RudyHackett_1977_NYN :: 28.9 :: False
EarlWilliams_1977_NYN :: 39.7 :: False

Outliers for FTR_advanced in season 1978 :
JimArd_1978_BOS :: 2.0 :: False

Outliers for TOV_per_100poss in season 1978 :
JohnJohnson_1978_HOU :: 12.9 :: False

Outliers for TOV_per_36min in season 1978 :
WilsonWashington_1978_PHI :: 9.5 :: False

Outliers for STL%_advanced in season 1979 :
GusGerard_1979_DET :: 14.9 :: False
BubblesHawkins_1979_DET :: 8.0 :: False

Outliers for FTA_per_100poss in season 1981 :
MontiDavis_1981_DAL :: 30.4 :: False

Outliers

In [20]:
len(data)

19127

## 3. Remove duplicated columns

In [22]:
# Two methods :
#data = data.T.drop_duplicates(how="all", keep="first").T
#data = data.drop(['MP_per_100poss'], axis='columns')

## Export

In [24]:
data.to_csv("./data/all_consolidated_final.csv", index=False)